<a href="https://colab.research.google.com/github/alihussainia/Love-Letter-Generator/blob/main/data_prep_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations

In [ ]:
%pip install beautifulsoup4 jsonlines

# Imports

In [ ]:
import pandas as pd
import jsonlines
import json
import urllib
from bs4 import BeautifulSoup

# Preprocessing



---



The results will be used to fine-tune GPT-J Models.

## First Site
https://relishbay.com/sweet-love-letters-for-girlfriend/

In [ ]:
URL = "https://relishbay.com/sweet-love-letters-for-girlfriend/"
raw = urllib.request.urlopen(URL).read()
original = raw.decode('utf-8')
soup = BeautifulSoup(original) 

In [ ]:
res = [p.get_text().strip() for p in soup.find_all('p')]

In [ ]:
initial_result = []
for i in range(len(res)):
  if i == len(res):
    break
  elif res[i] == "":
    print(res[i])
    res.remove(res[i])
  elif res[i][0].isnumeric():
    initial_result.append(res[i])

In [ ]:
dataset_girlfriend = []
for string in initial_result:
  for i in range(len(string)):
    if "\n" in string[i]:
      dataset_girlfriend.append(string[i+1:])
      break

## Second Site
https://happilylover.com/love-letters-for-girlfriend/

In [ ]:
URL = "https://happilylover.com/love-letters-for-girlfriend/"
raw = urllib.request.urlopen(URL).read()
original = raw.decode('utf-8')
soup = BeautifulSoup(original) 

In [ ]:
res = [p.get_text().strip() for p in soup.find_all('p')]

In [ ]:
initial_result = res[1:]

In [ ]:
dataset_girlfriend = []
for string in initial_result:
  for i in range(len(string)):
    if "," in string[i] and string[i+1] == " ":
      dataset_girlfriend.append(string)
    elif "," in string[i]:
      dataset_girlfriend.append(string[i+1:])
      break

## Third Site
https://memesbams.com/love-letters-for-her/

In [ ]:
URL = "https://memesbams.com/love-letters-for-her/"
#defining header
header= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
      'AppleWebKit/537.11 (KHTML, like Gecko) '
      'Chrome/23.0.1271.64 Safari/537.11',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
      'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
      'Accept-Encoding': '*',
      'Accept-Language': 'en-US,en;q=0.8',
      'Connection': 'keep-alive'}

#the URL where you are requesting at
req = urllib.request.Request(url=URL, headers=header) 
page = urllib.request.urlopen(req).read()

original = page.decode('utf-8')
soup = BeautifulSoup(original)

In [ ]:
res = [p.get_text().strip() for p in soup.find_all('li')]

In [ ]:
removal_begin = ['Relationships',
 'Quotes',
 'Birthday',
 'Holidays',
 'Memes & GIFs',
 'Jokes',
 'About Us',
 'Contact Us',
 'Terms of Service',
 'Privacy Policy',
 'Copyright',
 'Choose a deep love letter for your girlfriend',
 'Long, thoughtful and amazing love letters to my wife',
 'What about a cute "I love you" letter for your girlfriend?',
 'Need a romantic and emotional love letter for her from the heart?',
 'Probably the sweetest love letter for her is here!',
 'Really the best ideas of short love letters for her',
 'Love letters for her: tell her about missing her!',
 'Need a love letter for your girlfriend for anniversary?',
 "What about love letters for her on Valentine's day?",
 'Amazing & beautiful love letter to a girl to impress her',
 'Best Love Letters Images with Quotes',]

rb_len = len(removal_begin)

In [ ]:
removal_close = ['Mann, L. (2017, November 18). Why love letters matter, even after you break up. Chicago Tribune. https://www.chicagotribune.com/lifestyles/sc-fam-0609-love-letters-20150602-story.html',
 '\u200cArt. (2009, June 27). How to Write a Love Letter. The Art of Manliness. https://www.artofmanliness.com/articles/30-days-to-a-better-man-day-28-write-a-love-letter/',
 'Facebook',
 'Pinterest',
 '',
 '',
 '',
 '',
 '',
 'About Us',
 'Contact Us',
 'Partners',
 'Editorial Guidelines',
 'Terms of Service',
 'Privacy Policy',
 'Copyright',
 '',
 '',
 '']

rc_len = len(removal_close)

In [ ]:
res = res[rb_len:]
final_len = len(res) - rc_len
initial_result = res[:final_len]

In [ ]:
dataset_girlfriend = []
for string in initial_result:
  for i in range(len(string)):
    if "," in string[i] and string[i+1] == " ":
      dataset_girlfriend.append(string)
    elif "," in string[i]:
      dataset_girlfriend.append(string[i+1:])
      break

# Saving

In [ ]:
# Updating the wife_dataset file with new wife love letter sample(s)
map_dict = {'<dataset>':dataset_girlfriend}
txt = "dataset = <dataset>"
with open('love_letter_dataset.py', 'w') as f:
    for k,v in map_dict.items():
      txt = txt.replace(k,str(v))
    f.write(txt)

In [ ]:
outer_list = []
for i in range(len(dataset_girlfriend)): # change dataset name
    inner_dict = {}
    inner_dict["input"] =  "love letter" # change variable name
    inner_dict["output"] =  dataset_girlfriend[i] # change dataset name
    outer_list.append(inner_dict)

# removing duplicates
outer_list = [i for n, i in enumerate(outer_list) if i not in outer_list[:n]] 

In [ ]:
# saving to json format
with open('love_letter_dataset.json', 'w', encoding='utf8') as json_output:
  json.dump(outer_list, json_output, ensure_ascii=False)

In [ ]:
# saving to jsonl format
with open('love_letter_dataset.jsonl', 'w', encoding='utf8') as jsonl_output:
    for entry in outer_list:
        json.dump(entry, jsonl_output, ensure_ascii=False)

        jsonl_output.write('\n')